In [191]:
import pandas as pd
tekme_redne = pd.read_csv('statistika-v-csv/tekme_redne.csv', index_col='Tekma',encoding = "utf-8")
tekme_playoff = pd.read_csv('statistika-v-csv/tekme_playoff.csv', index_col='Tekma',encoding = "utf-8")
ekipe = pd.read_csv('statistika-v-csv/ekipe.csv', index_col='id',encoding = "utf-8")
pd.options.display.max_rows = 10
%matplotlib notebook

In [192]:
tekme_redne = tekme_redne[tekme_redne.Team != 31] #izbrišemo All-Star tekmo
tekme_redne = tekme_redne[tekme_redne.Team != 32]
ekipe = ekipe.drop([31,32]) #izbrišemo All-Star ekipi
tekme_redne[["FGM", "FGA"]] = tekme_redne["FG"].str.split("-", expand = True)
tekme_redne[["3PM", "3PA"]] = tekme_redne["3PT"].str.split("-", expand = True)
tekme_redne[["FTM", "FTA"]] = tekme_redne["FT"].str.split("-", expand = True)
tekme_playoff[["FGM", "FGA"]] = tekme_playoff["FG"].str.split("-", expand = True)
tekme_playoff[["3PM", "3PA"]] = tekme_playoff["3PT"].str.split("-", expand = True)
tekme_playoff[["FTM", "FTA"]] = tekme_playoff["FT"].str.split("-", expand = True)
tekme_redne = tekme_redne.drop(columns=['FG','3PT','FT'])
tekme_playoff = tekme_playoff.drop(columns=['FG','3PT','FT'])


In [193]:
tekme_redne = tekme_redne.astype(float)
tekme_playoff = tekme_playoff.astype(float)

In [194]:
tekme_redne["FG%"] = 100 * tekme_redne["FGM"]/tekme_redne["FGA"]
tekme_redne["3P%"] = 100 * tekme_redne["3PM"]/tekme_redne["3PA"]
tekme_redne["FT%"] = 100 * tekme_redne["FTM"]/tekme_redne["FTA"]
tekme_playoff["FG%"] = 100 * tekme_playoff["FGM"]/tekme_playoff["FGA"]
tekme_playoff["3P%"] = 100 * tekme_playoff["3PM"]/tekme_playoff["3PA"]
tekme_playoff["FT%"] = 100 * tekme_playoff["FTM"]/tekme_playoff["FTA"]

In [195]:
tekme_redne["FG%"] = tekme_redne["FG%"].round(decimals=2)
tekme_redne["3P%"] = tekme_redne["3P%"].round(decimals=2)
tekme_redne["FT%"] = tekme_redne["FT%"].round(decimals=2)
tekme_playoff["FG%"] = tekme_playoff["FG%"].round(decimals=2)
tekme_playoff["3P%"] = tekme_playoff["3P%"].round(decimals=2)
tekme_playoff["FT%"] = tekme_playoff["FT%"].round(decimals=2)

In [196]:
zmaga_poraz_redne = []

for i in range(tekme_redne["PTS"].size):
    if i % 2 == 0:
        if tekme_redne["PTS"].iloc[i] > tekme_redne["PTS"].iloc[i + 1]:
            zmaga_poraz_redne.append(1)
        else:
            zmaga_poraz_redne.append(0)
    else:
        if tekme_redne["PTS"].iloc[i] > tekme_redne["PTS"].iloc[i - 1]:
            zmaga_poraz_redne.append(1)
        else:
            zmaga_poraz_redne.append(0)

tekme_redne["W/L"] = zmaga_poraz_redne

zmaga_poraz_playoff = []

for i in range(tekme_playoff["PTS"].size):
    if i % 2 == 0:
        if tekme_playoff["PTS"].iloc[i] > tekme_playoff["PTS"].iloc[i + 1]:
            zmaga_poraz_playoff.append(1)
        else:
            zmaga_poraz_playoff.append(0)
    else:
        if tekme_playoff["PTS"].iloc[i] > tekme_playoff["PTS"].iloc[i - 1]:
            zmaga_poraz_playoff.append(1)
        else:
            zmaga_poraz_playoff.append(0)

tekme_playoff["W/L"] = zmaga_poraz_playoff
tekme_playoff

,Team,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,FGM,FGA,3PM,3PA,FTM,FTA,FG%,3P%,FT%,W/L
Tekma,,,,,,,,,,,,,,,,,,,,
0,9.0,12.0,39.0,51.0,22.0,8.0,3.0,17.0,23.0,107.0,36.0,99.0,20.0,50.0,15.0,19.0,36.36,40.00,78.95,0
0,29.0,15.0,49.0,64.0,19.0,11.0,1.0,17.0,18.0,109.0,42.0,96.0,5.0,31.0,20.0,33.0,43.75,16.13,60.61,1
1,28.0,10.0,32.0,42.0,22.0,2.0,3.0,12.0,21.0,113.0,38.0,76.0,17.0,36.0,20.0,26.0,50.00,47.22,76.92,1
1,3.0,13.0,26.0,39.0,22.0,7.0,7.0,5.0,21.0,103.0,37.0,84.0,11.0,40.0,18.0,24.0,44.05,27.50,75.00,0
2,24.0,10.0,29.0,39.0,29.0,3.0,2.0,6.0,14.0,123.0,43.0,91.0,19.0,40.0,18.0,19.0,47.25,47.50,94.74,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,29.0,17.0,31.0,48.0,22.0,11.0,3.0,5.0,18.0,109.0,39.0,97.0,7.0,29.0,24.0,29.0,40.21,24.14,82.76,1
86,29.0,11.0,26.0,37.0,26.0,7.0,1.0,11.0,17.0,123.0,50.0,87.0,14.0,28.0,9.0,17.0,57.47,50.00,52.94,1
86,27.0,8.0,27.0,35.0,23.0,9.0,5.0,8.0,20.0,119.0,48.0,87.0,13.0,19.0,10.0,11.0,55.17,68.42,90.91,0


In [197]:
ekipe_redno_2 = tekme_redne.groupby("Team")
ekipe_playoff_2 = tekme_playoff.groupby("Team")
ekipe_redno = ekipe_redno_2.sum()
ekipe_playoff = ekipe_playoff_2.sum()

In [198]:
ekipe_redno["FG%"] = ekipe_redno["FG%"]/ekipe_redno_2.count()["FG%"]
ekipe_redno["3P%"] = ekipe_redno["3P%"]/ekipe_redno_2.count()["3P%"]
ekipe_redno["FT%"] = ekipe_redno["FT%"]/ekipe_redno_2.count()["FT%"]
ekipe_playoff["FG%"] = ekipe_playoff["FG%"]/ekipe_playoff_2.count()["FG%"]
ekipe_playoff["3P%"] = ekipe_playoff["3P%"]/ekipe_playoff_2.count()["3P%"]
ekipe_playoff["FT%"] = ekipe_playoff["FT%"]/ekipe_playoff_2.count()["FT%"]
ekipe_redno["3PA/FGA"] = ekipe_redno["3PA"]/ekipe_redno["FGA"]
ekipe_playoff["3PA/FGA"] = ekipe_playoff["3PA"]/ekipe_playoff["FGA"]

In [199]:
ekipe_redno = ekipe_redno.rename(columns={"W/L" : "WINS"})
ekipe_playoff =ekipe_playoff.rename(columns={"W/L" : "WINS"})
ekipe_redno["WIN%"] = 100 * ekipe_redno["WINS"]/ekipe_redno_2.count()["W/L"]
ekipe_playoff["WIN%"] = 100 * ekipe_playoff["WINS"]/ekipe_playoff_2.count()["W/L"]

In [200]:
lestvica_redno = ekipe_redno.sort_values(by='WIN%',ascending=False)
najvec_relativno_za_tri_redno = ekipe_redno.sort_values(by='3PA/FGA',ascending=False)

In [202]:
lestvica_redno

,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,FGM,...,3PM,3PA,FTM,FTA,FG%,3P%,FT%,WINS,3PA/FGA,WIN%
Team,,,,,,,,,,,,,,,,,,,,,
23.0,765.0,2708.0,3473.0,1703.0,475.0,370.0,973.0,1335.0,8382.0,2971.0,...,1205.0,3098.0,1235.0,1546.0,46.914167,38.772917,79.325139,52,0.488335,72.222222
27.0,613.0,2380.0,2993.0,1882.0,505.0,298.0,838.0,1335.0,8071.0,3027.0,...,916.0,2425.0,1101.0,1316.0,49.142429,37.759000,83.673714,49,0.392840,70.000000
12.0,722.0,2521.0,3243.0,1706.0,655.0,447.0,988.0,1456.0,8182.0,2981.0,...,811.0,2169.0,1409.0,1836.0,47.688611,37.599444,76.752778,49,0.346652,68.055556
22.0,749.0,2419.0,3168.0,1908.0,575.0,322.0,910.0,1354.0,8176.0,3077.0,...,916.0,2426.0,1106.0,1375.0,48.666197,37.876901,80.862958,47,0.382589,66.197183
2.0,620.0,2479.0,3099.0,1877.0,471.0,365.0,910.0,1327.0,8287.0,3019.0,...,990.0,2535.0,1259.0,1568.0,49.445143,38.684429,80.265143,46,0.414013,65.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6.0,751.0,2328.0,3079.0,1716.0,559.0,325.0,1039.0,1308.0,7476.0,2778.0,...,720.0,2142.0,1200.0,1614.0,45.044028,33.663194,74.271250,22,0.346826,30.555556
25.0,705.0,2531.0,3236.0,1556.0,496.0,316.0,1095.0,1291.0,7446.0,2750.0,...,842.0,2493.0,1104.0,1520.0,44.150423,33.838732,72.642113,21,0.399263,29.577465
10.0,746.0,2527.0,3273.0,1572.0,495.0,320.0,882.0,1235.0,7488.0,2754.0,...,784.0,2287.0,1196.0,1543.0,42.935556,33.912361,77.584444,21,0.356175,29.166667
